In [ ]:
# EvalHarness runs evals asychronously, so we need to install nest_asyncio to avoid errors
# running the evals in a notebook
!pip install nest_asyncio

In [1]:
import time
from llm_evals.eval import EvalHarness, EvalResult
import nest_asyncio
nest_asyncio.apply()

def print_result(result: EvalResult) -> None:
    """Print the result of an evaluation via callback."""
    print(result)

eval_harness = EvalHarness(callback=print_result)
eval_harness.add_eval_from_yaml('../examples/evals/simple_example.yaml')
eval_harness.add_eval_from_yaml('../examples/evals/mask_emails.yaml')
eval_harness.add_candidate_from_yaml('../examples/candidates/openai_3.5_1106.yaml')
eval_harness.add_candidate_from_yaml('../examples/candidates/openai_4.0_1106.yaml')

print('Starting eval_harness')
start = time.time()
results = eval_harness()
end = time.time()
print(f"Total time: {end - start}")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting eval_harness
EvalResult:
    Candidate:                  OpenAI GPT-3.5-Turbo (1106)
    Eval:                       Fibonacci Sequence
    # of Prompts Tested:        2
    Cost:                       $0.0011
    Total Response Time:        9.5 seconds
    # of Response Characters:   1,267
    # of Code Blocks Generated: 3
    Characters per Second:      133.3
    # of Checks:                5
    # of Successful Checks:     4
    % of Successful Checks:     80.0%
EvalResult:
    Candidate:                  OpenAI GPT-3.5-Turbo (1106)
    Eval:                       Python Function to Mask Emails
    # of Prompts Tested:        2
    Cost:                       $0.0010
    Total Response Time:        10.4 seconds
    # of Response Characters:   1,307
    # of Code Blocks Generated: 2
    Characters per Second:      125.9
    # of Checks:                6
    # of Successful Checks:     5
    % of Successful Checks:     83.3%
EvalResult:
    Candidate:                  OpenAI 

---

In [7]:
import time
from llm_evals.eval import EvalHarness, EvalResult
import nest_asyncio
nest_asyncio.apply()

def print_result(result: EvalResult) -> None:
    """Print the result of an evaluation via callback."""
    print(result)

eval_harness = EvalHarness(callback=print_result)
eval_harness.add_evals_from_yamls('../examples/evals/*.yaml')
# includes a Hugging Face Endpoint candidate and requires valid Endpoint and API key
eval_harness.add_candidate_from_yaml('../examples/candidates/hugging_face_endpoint_mistral_a10g.yaml')
# eval_harness.add_candidates_from_yamls('../examples/candidates/*.yaml')

print('Starting eval_harness')
start = time.time()
results = eval_harness()
end = time.time()
print(f"Total time: {end - start}")

Starting eval_harness


AttributeError: Can't pickle local object 'create_message_formatter.<locals>.message_formatter'

---